### Required Libraries

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


### Load Dataset

In [2]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [3]:
#Create New Target (Good in 60-15 but bad in 90-24)
model_data['G15_B24'] = model_data.apply(
   lambda row: 0 if (row['60_in_15mob'] == 0 and row['90_in_24mob'] == 0)
    else ( 1 if (row['60_in_15mob'] == 0 and row['90_in_24mob'] == 1)
    else 2),axis=1)

model_data= model_data[model_data['G15_B24']!=2]
print('********* New Target Prepared *********')

********* New Target Prepared *********


In [5]:
model_data.shape

(47632, 2832)

### Drop Null target records

In [6]:
model_data['G15_B24'].value_counts(dropna=False)

G15_B24
0    44460
1     3172
Name: count, dtype: int64

In [7]:
df = model_data.dropna(subset=['G15_B24'])

In [8]:
df['G15_B24'].value_counts(dropna=False)

G15_B24
0    44460
1     3172
Name: count, dtype: int64

In [9]:
df['user_segment'].value_counts()

user_segment
OFFUS    33813
ONUS     13819
Name: count, dtype: int64

In [10]:
# take one copy for model development data
data = df.copy()

In [11]:
#Handling infinity and replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

### Split the Time interval

In [ ]:
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2022-06')]
oot_df1= data[(data['month_year']>='2022-07') & (data['month_year']<='2022-09')]

In [ ]:
model_df1.shape, oot_df1.shape

In [20]:
features_list=[
    'bounced_transactions_count',
'Min_Balance_Min_M1_M2_M3',
'utilities_spending_pct',
'cash_withdrawal_sum',
'upi_trans_count',
'num_credits_last_1_months',
'ratio_balance_after_2_days_max_credit_M3',
'ratio_6th_day_balance_to_max_balance_M3',
'range_max_credit_last_2_months',
'num_upi_trans_last_3_months',
'ratio_digital_vs_withdrawal',
'balance_5th_day_M1',
'time_max_to_70pct_balance_M2',
'min_max_credit_last_3_months',
'ratio_total_debit_to_total_credit_M2',
'time_between_max_min_balance_M2',
'month_year',
 'G15_B24']

In [21]:
# Selected Features for model build purpose
model_df = model_df1[features_list]

In [22]:
oot_df = oot_df1[features_list]

In [23]:
model_df.shape, oot_df.shape

((34344, 18), (10802, 18))

In [24]:
model_df['month_year'].isnull().sum()

0

In [25]:
#Handling Null Values
for i in model_df.columns:
    model_df[i].fillna(-9999,inplace=True)

/tmp/ipykernel_2877537/2889689820.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  model_df[i].fillna(-9999,inplace=True)
/tmp/ipykernel_2877537/2889689820.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df[i].fillna(-9999,inplace=True)


In [26]:
#Handling Null Values
for i in oot_df.columns:
    oot_df[i].fillna(-9999,inplace=True)

/tmp/ipykernel_2877537/1074864086.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  oot_df[i].fillna(-9999,inplace=True)
/tmp/ipykernel_2877537/1074864086.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oot_df[i].fillna(-9999,inplace=True)


In [27]:
model_df['G15_B24'].value_counts(dropna=False)

G15_B24
0    32193
1     2151
Name: count, dtype: int64

In [28]:
oot_df['G15_B24'].value_counts(dropna=False)

G15_B24
0    9980
1     822
Name: count, dtype: int64

In [29]:
model_df = model_df[(model_df !=-9999).all(axis=1)]

In [30]:
model_df.shape

(4914, 18)

In [31]:
oot_df = oot_df[(oot_df !=-9999).all(axis=1)]


In [32]:
oot_df.shape

(1746, 18)

In [33]:
# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', 'G15_B24']) # Adjust 'target' to your actual target column name
y_model = model_df['G15_B24']

In [34]:
X_oot = oot_df.drop(columns=['month_year', 'G15_B24'])
y_oot = oot_df['G15_B24']

In [35]:
X_oot.shape,y_oot.shape

((1746, 16), (1746,))

In [36]:
X_model.shape

(4914, 16)

### Split Model dataset into Train and Test

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [38]:
X_train.shape, X_test.shape

((2948, 16), (1966, 16))

In [39]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(2358, 16) (590, 16) (1966, 16)


In [40]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new)
deval = xgb.DMatrix(X_eval, label=y_eval)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [41]:
# params = {
#     'objective': 'binary:logistic',
#     'eval_metric': 'auc',
#     'learning_rate': 0.01,
#      'max_depth': 2,
#     'subsample': 0.6,
#     'colsample_bytree': 0.6,

#     'lambda': 6.9,
#     'alpha': 4.6,
#     'gamma': 5.2,
   
#     'scale_pos_weight': 3.6,
#     'random_state': 24
    
# }

# evals = [(dtrain, 'train'), (dtest, 'eval')]
# xgb_model = xgb.train(
#     params,
#     dtrain,
#     num_boost_round= 1111,
#     early_stopping_rounds=40,
#     evals=evals,
#     verbose_eval=10
# )

In [42]:
# params = {
#     'objective': 'binary:logistic',
#     'eval_metric': 'auc',
#     'learning_rate': 0.01,
#      'max_depth': 2,

#     'subsample': 0.4,
#     'colsample_bytree': 0.4,
#     # 'min_child_weight':25,

#     #'lambda': 2.6,
#     'lambda': 5.5,
#     'alpha':5.0,
#     'gamma':4.5,
#     # 'scale_pos_weight':3.0,
#     'random_state': 42
    
# }

params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.1,
    'max_depth': 2,
    'subsample': 0.4,
    'colsample_bytree': 0.4,
    # 'min_child_weight':30,
    # 'lambda': 10,
    'lambda': 4.7,
    'alpha': 4.8,
    'gamma': 5.6,
    #'scale_pos_weight': 1,
    'random_state': 24
}

evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 70,
    early_stopping_rounds= 40,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.54640	eval-auc:0.53282
[10]	train-auc:0.62335	eval-auc:0.55054
[20]	train-auc:0.62837	eval-auc:0.54801
[30]	train-auc:0.62639	eval-auc:0.54563
[40]	train-auc:0.62639	eval-auc:0.54563
[45]	train-auc:0.62635	eval-auc:0.55023


In [43]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.63, Gini: 0.25, KS: 0.22
Eval AUC: 0.55, Gini: 0.10, KS: 0.14
Test AUC: 0.58, Gini: 0.16, KS: 0.15
OOT AUC: 0.60, Gini: 0.19, KS: 0.16


In [161]:
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.4f}, KS: {train_ks:.4f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.4f}, KS: {test_ks:.4f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.4f}, KS: {oot_ks:.4f}")

Train AUC: 0.63, Gini: 0.2527, KS: 0.2172
Test AUC: 0.58, Gini: 0.1621, KS: 0.1500
OOT AUC: 0.60, Gini: 0.1923, KS: 0.1604


In [44]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                       Feature  Importance
4  num_upi_trans_last_3_months         3.0
0   bounced_transactions_count         2.0
2              upi_trans_count         2.0
1          cash_withdrawal_sum         1.0
3    num_credits_last_1_months         1.0


In [45]:
feature_importance_df

Feature  Importance
4  num_upi_trans_last_3_months         3.0
0   bounced_transactions_count         2.0
2              upi_trans_count         2.0
1          cash_withdrawal_sum         1.0
3    num_credits_last_1_months         1.0

In [46]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
6          0.144068         0.101707        0.112355        0.094364    236   
5          0.113300         0.088027        0.093540        0.084133    203   
4          0.104478         0.081135        0.083846        0.076879    268   
3          0.070312         0.076273        0.076614        0.076219    128   
2          0.079585         0.074380        0.076147        0.072994    289   
1          0.046191         0.069685        0.072612        0.069167   1234   

        label_good  label_bad  
Decile                         
6              202         34  
5              180         23  
4              240         28  
3              119          9  
2              266         23  
1             1177         57

In [47]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
6          0.092308         0.100751        0.112355        0.093540    195   
5          0.074534         0.088045        0.091956        0.084133    161   
4          0.070175         0.081136        0.083846        0.076879    228   
3          0.040816         0.076283        0.076614        0.076219     98   
2          0.093117         0.074606        0.076147        0.072994    247   
1          0.044359         0.069697        0.072612        0.069167   1037   

        label_good  label_bad  
Decile                         
6              177         18  
5              149         12  
4              212         16  
3               94          4  
2              224         23  
1              991         46

In [48]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
7          0.149425         0.104150        0.112355        0.096889    174   
6          0.160000         0.091620        0.096418        0.086352    175   
5          0.077844         0.084094        0.086166        0.080679    167   
4          0.106250         0.079311        0.080603        0.076482    160   
3          0.098958         0.076177        0.076219        0.075821    192   
2          0.067308         0.073236        0.074166        0.072994    104   
1          0.067183         0.069645        0.072612        0.069167    774   

        label_good  label_bad  
Decile                         
7              148         26  
6              147         28  
5              154         13  
4              143         17  
3              173         19  
2               97          7  
1              722         52

In [49]:
xgb_model.save_model('LeoPayu_BA_G15_B24_Final.json')
print('Model Saved')

Model Saved
